In [18]:
import numpy as np
import pandas as pd
import random
import sklearn.linear_model
import statsmodels.api as sm
import statsmodels

# Problem 2

## Part (b)

### Generate Data

In [19]:
n, ps, rhos = 100, [10, 25, 50], [0, 0.25, 0.5]

In [20]:
def generate_data(n, p, rho, signal='sparse'):
    if signal == 'sparse':
        beta = np.array([[2/np.sqrt(n) * (j <= np.sqrt(p)) for j in range(1,p+1,1)]]).T
    elif signal == 'dense':
        beta = np.array([[5/(j*np.sqrt(n)) for j in range(1,p+1,1)]]).T
    else:
        return
    R2 = 0.8
    sigma_rho = np.reshape([rho**(np.abs(i-j)) for i in range(1,p+1,1) for j in range(1,p+1,1)], (-1, p))
    sigma2 = ((1-R2)/R2 * (beta.T) @ sigma_rho @ beta)[0][0]
    sigma = np.sqrt(sigma2)
    epsilon = np.random.normal(0, sigma, n).reshape(n,1)
    X = np.random.multivariate_normal(np.zeros(p), sigma_rho, n)
    Y = (X @ beta + epsilon).flatten()
    return X, Y

In [57]:
# Example
# X, Y = generate_data(100, 10, 0, 'sparse')
# X.shape, Y.shape

In [58]:
# Example
# X, Y = generate_data(100, 10, 0, 'dense')
# X.shape, Y.shape

### LASSO - AIC/BIC/LOO-CV

In [331]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoLarsIC
from sklearn.pipeline import make_pipeline

from sklearn.linear_model import LassoCV

from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

# Code source: 
# https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_model_selection.html 
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html#sklearn.linear_model.LassoCV
# https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_model_selection.html

def fit_LASSO(X, Y):
    # Tune alpha
    
    ## AIC:
    lasso_lars_ic = make_pipeline(
        StandardScaler(), LassoLarsIC(criterion="aic", normalize=False)
    ).fit(X, Y)
    
    alpha_aic = lasso_lars_ic[-1].alpha_
    
    ## BIC:
    lasso_lars_ic.set_params(lassolarsic__criterion="bic").fit(X, Y)
    #results["BIC criterion"] = lasso_lars_ic[-1].criterion_
    alpha_bic = lasso_lars_ic[-1].alpha_
    
    ## LOO-CV:
    ### LOO-CV is just k-fold CV with k = number of observations (n)
    lasso_model = make_pipeline(StandardScaler(), LassoCV(cv=n, n_alphas=20)).fit(X, Y)
    lasso = lasso_model[-1]
    alpha_cv = lasso.alpha_
    
    # Fit LASSO with tuned alpha:
    
    ## with alpha from AIC:
    clf = Lasso(alpha=alpha_aic)
    clf.fit(X, Y)
    Y_pred = clf.predict(X)
    mse_aic = mean_squared_error(Y, Y_pred)
    
    ## with alpha from BIC:
    clf = Lasso(alpha=alpha_aic)
    clf.fit(X, Y)
    Y_pred = clf.predict(X)
    mse_bic = mean_squared_error(Y, Y_pred)
    
    ## with alpha from LOO-CV:
    clf = Lasso(alpha=alpha_cv)
    clf.fit(X, Y)
    Y_pred = clf.predict(X)
    mse_cv = mean_squared_error(Y, Y_pred)
    
    return [mse_aic, mse_bic, mse_cv]

In [330]:
def fit_model(n, p, rho, signal):
    data_list = [generate_data(n, p, rho, signal) for i in range(1000)]
    mse_ls = np.array([fit_LASSO(data_list[i][0],data_list[i][1]) for i in range(1000)])
    return np.mean(mse_ls, axis = 0)

In [236]:
import time
start_time = time.time()
result_sparse = [fit_model(n, p, rho, 'sparse') for p in ps for rho in rhos]
end_time = time.time()
run_time = end_time - start_time

In [336]:
# Run time in minutes
run_time / 60

37.61602555116018

In [240]:
start_time = time.time()
result_dense = [fit_model(n, p, rho, 'dense') for p in ps for rho in rhos]
end_time = time.time()
run_time2 = end_time - start_time

In [335]:
# Run time in minutes
run_time2 / 60

36.08029566208521

In [308]:
result_df_sparse = pd.DataFrame(result_sparse, columns = ['AIC.MSE', 'BIC.MSE', 'LOOCV.MSE'])
result_df_dense = pd.DataFrame(result_dense, columns = ['AIC.MSE', 'BIC.MSE', 'LOOCV.MSE'])

result_df = pd.concat([result_df_sparse, result_df_dense])
p_val = np.array([[p,rho] for p in ps for rho in rhos]*2)[:,0]
rho_val = np.array([[p,rho] for p in ps for rho in rhos]*2)[:,1]

result_df['p'] = p_val
result_df['rho'] = rho_val
result_df['Estimator'] = ['LASSO']*result_df.shape[0]
result_df['Signal'] = ['Sparse']*(int(result_df.shape[0]/2))+['Dense']*(int(result_df.shape[0]/2))
result_df = result_df[['p','rho','Estimator','Signal','AIC.MSE', 'BIC.MSE', 'LOOCV.MSE']]
result_df.reset_index(inplace=True, drop=True)

In [309]:
result_df

,p,rho,Estimator,Signal,AIC.MSE,BIC.MSE,LOOCV.MSE
0,10.0,0.00,LASSO,Sparse,0.027770,0.027770,0.027783
1,10.0,0.25,LASSO,Sparse,0.038466,0.038466,0.038554
2,10.0,0.50,LASSO,Sparse,0.050768,0.050768,0.050914
3,25.0,0.00,LASSO,Sparse,0.042836,0.042836,0.043202
4,25.0,0.25,LASSO,Sparse,0.064751,0.064751,0.065386
5,25.0,0.50,LASSO,Sparse,0.096796,0.096796,0.097836
6,50.0,0.00,LASSO,Sparse,0.053860,0.053860,0.054426
7,50.0,0.25,LASSO,Sparse,0.084834,0.084834,0.086605
8,50.0,0.50,LASSO,Sparse,0.137249,0.137249,0.140839
9,10.0,0.00,LASSO,Dense,0.086454,0.086454,0.086365


### Ridge - AIC/BIC/LOO-CV

In [9]:
alpha_array = np.arange(0.001, 0.1, 0.001)

def Ridge(tuning = 'aic', data = 'sparse'):
    
    MSE_mean = []
    for p in ps:
        for rho in rhos:
            MSE = []
            for i in range(1000):
                X, Y = generate_data(100, p, rho, data)

                # Standardize each column and add constant column
                X_std = (X - X.mean(axis=0)) / X.std(axis=0)
                X_std = sm.add_constant(X_std)

                # Set OLS model without regularization
                model = sm.OLS(Y, X_std)
                results_fu = model.fit()
                standard = []

                for n in alpha_array.tolist():
                    results_fr = model.fit_regularized(L1_wt=0, alpha=n, start_params=results_fu.params)
                    final = sm.regression.linear_model.OLSResults(model, 
                                                                  results_fr.params, 
                                                                  model.normalized_cov_params)
                    if tuning == 'aic':
                        standard.append(final.aic)

                    elif tuning == 'bic':
                        standard.append(final.bic)

                optimal_alpha = alpha_array[standard == min(standard)][0]

                # Fit the optimal model for the dataset
                results_opt = model.fit_regularized(L1_wt=0, alpha=optimal_alpha, start_params=results_fu.params)
                Y_pred = results_opt.predict(X_std)
                MSE.append(np.mean((Y - Y_pred)**2))

            MSE_mean.append(np.mean(MSE))
            
    return(MSE_mean)

In [21]:
from sklearn.linear_model import RidgeCV

alpha_array = np.arange(0.001, 0.1, 0.001)

def Ridge_CV(data = 'sparse'):
    
    MSE_mean = []
    for p in ps:
        for rho in rhos:
            MSE = []
            for i in range(1000):
                X, Y = generate_data(100, p, rho, data)

                # Standardize each column and add constant column
                X_std = (X - X.mean(axis=0)) / X.std(axis=0)
                X_std = sm.add_constant(X_std)

                # Set OLS model without regularization
                model = sm.OLS(Y, X_std)
                results_fu = model.fit()
                standard = []
                
                # LOOCV
                ridge_cv = RidgeCV(alphas = alpha_array, store_cv_values = True).fit(X_std, Y)
                standard = ridge_cv.cv_values_.mean(axis=-0)
                optimal_alpha = alpha_array[standard == min(standard)][0]

                # Fit the optimal model for the dataset
                results_opt = model.fit_regularized(L1_wt=0, alpha=optimal_alpha, start_params=results_fu.params)
                Y_pred = results_opt.predict(X_std)
                MSE.append(np.mean((Y - Y_pred)**2))

            MSE_mean.append(np.mean(MSE))
            
    return(MSE_mean)

In [22]:
sparse_loocv = Ridge_CV(data = 'sparse')

In [24]:
dense_loocv = Ridge_CV(data = 'dense')

In [34]:
sparse_loocv

[0.027932621447725077,
 0.037696108457427824,
 0.04948046017765128,
 0.039095679793413635,
 0.05633835913639096,
 0.08443814592466029,
 0.03859972452976537,
 0.05710723781045427,
 0.09026683768981418]

In [25]:
dense_loocv

[0.08964888458111411,
 0.1203707334656788,
 0.16812632935435012,
 0.07852196719760775,
 0.10547529604894455,
 0.1514283950107423,
 0.05583672546509562,
 0.07433065303887218,
 0.108823686502854]

In [26]:
sparse_aic = Ridge(tuning = 'aic', data = 'sparse')

In [27]:
sparse_aic

[0.026473577693344016,
 0.03670380612665447,
 0.04857600247799384,
 0.03725844509196598,
 0.05475929463713193,
 0.08261866452262609,
 0.034465973160211265,
 0.05269265145521641,
 0.08306065501059798]

In [28]:
dense_aic = Ridge(tuning = 'aic', data = 'dense')

In [29]:
dense_aic

[0.08629634572385018,
 0.11737408612034608,
 0.1655006815304308,
 0.07438165676043129,
 0.10193254335404156,
 0.14630276782366838,
 0.05010210767288612,
 0.06854494923967634,
 0.09891384443314946]

In [30]:
sparse_bic = Ridge(tuning = 'bic', data = 'sparse')

In [31]:
sparse_bic

[0.02679961791286217,
 0.03674915616228412,
 0.04904656235319827,
 0.036857605133088424,
 0.055953026735223094,
 0.08187077522600518,
 0.03433842276452399,
 0.05266112461957276,
 0.08348684558478885]

In [32]:
dense_bic = Ridge(tuning = 'bic', data = 'dense')

In [33]:
dense_bic

[0.08564738827172025,
 0.11670948166762439,
 0.16507508856695008,
 0.07387224653924812,
 0.10134871056507626,
 0.14642930851727715,
 0.04976185594232425,
 0.06832147017686839,
 0.09920082453528757]

In [40]:
AIC = sparse_aic + dense_aic
BIC = sparse_bic + dense_bic
loocv = sparse_loocv + dense_loocv

In [41]:
data = {'AIC.MSE': AIC,
       'BIC.MSE': BIC,
       'LOOCV.MSE': loocv}

In [42]:
result_df_ridge = pd.DataFrame(data)

In [43]:
p_val = np.array([[p,rho] for p in ps for rho in rhos]*2)[:,0]
rho_val = np.array([[p,rho] for p in ps for rho in rhos]*2)[:,1]

result_df_ridge['p'] = p_val
result_df_ridge['rho'] = rho_val
result_df_ridge['Estimator'] = ['Ridge']*result_df_ridge.shape[0]
result_df_ridge['Signal'] = ['Sparse']*(int(result_df_ridge.shape[0]/2))+['Dense']*(int(result_df_ridge.shape[0]/2))
result_df_ridge = result_df_ridge[['p','rho','Estimator','Signal','AIC.MSE', 'BIC.MSE', 'LOOCV.MSE']]
result_df_ridge.reset_index(inplace=True, drop=True)

In [47]:
result_df_ridge

,p,rho,Estimator,Signal,AIC.MSE,BIC.MSE,LOOCV.MSE
0,10.0,0.00,Ridge,Sparse,0.026474,0.026800,0.027933
1,10.0,0.25,Ridge,Sparse,0.036704,0.036749,0.037696
2,10.0,0.50,Ridge,Sparse,0.048576,0.049047,0.049480
3,25.0,0.00,Ridge,Sparse,0.037258,0.036858,0.039096
4,25.0,0.25,Ridge,Sparse,0.054759,0.055953,0.056338
5,25.0,0.50,Ridge,Sparse,0.082619,0.081871,0.084438
6,50.0,0.00,Ridge,Sparse,0.034466,0.034338,0.038600
7,50.0,0.25,Ridge,Sparse,0.052693,0.052661,0.057107
8,50.0,0.50,Ridge,Sparse,0.083061,0.083487,0.090267
9,10.0,0.00,Ridge,Dense,0.086296,0.085647,0.089649


In [50]:
result_df_ridge.to_csv('Ridge_results.csv')

In [61]:
Lasso_results = pd.read_csv('lasso_result.csv', index_col = 0)
Ridge_results = pd.read_csv('Ridge_results.csv', index_col = 0)
AdaLasso_results = pd.read_csv('result_adaptive_lasso.csv', index_col = 0)
AdaRidge_results = pd.read_csv('result.csv', index_col = 0)
AdaLasso_results = AdaLasso_results[1:]

In [62]:
Lasso_results

,p,rho,Estimator,Signal,AIC.MSE,BIC.MSE,LOOCV.MSE
0,10.0,0.00,LASSO,Sparse,0.027627,0.027627,0.027703
1,10.0,0.25,LASSO,Sparse,0.037986,0.037986,0.038106
2,10.0,0.50,LASSO,Sparse,0.051029,0.051029,0.051203
3,25.0,0.00,LASSO,Sparse,0.042818,0.042818,0.043129
4,25.0,0.25,LASSO,Sparse,0.063914,0.063914,0.064389
5,25.0,0.50,LASSO,Sparse,0.097499,0.097499,0.098773
6,50.0,0.00,LASSO,Sparse,0.053266,0.053266,0.054232
7,50.0,0.25,LASSO,Sparse,0.082844,0.082844,0.085231
8,50.0,0.50,LASSO,Sparse,0.136257,0.136257,0.139159
9,10.0,0.00,LASSO,Dense,0.087002,0.087002,0.086930


In [66]:
AdaLasso_results = AdaLasso_results.reset_index(drop=True)

In [67]:
AdaLasso_results

,p,rho,Estimator,Signal,AIC.MSE,BIC.MSE,LOOCV.MSE
0,10.0,0.00,Adaptive_Lasso,Sparse,0.027922,0.028848,0.028201
1,10.0,0.25,Adaptive_Lasso,Sparse,0.038535,0.039661,0.038869
2,10.0,0.50,Adaptive_Lasso,Sparse,0.051576,0.053011,0.052031
3,25.0,0.00,Adaptive_Lasso,Sparse,0.043891,0.047647,0.045368
4,25.0,0.25,Adaptive_Lasso,Sparse,0.064838,0.069760,0.067087
5,25.0,0.50,Adaptive_Lasso,Sparse,0.099665,0.105163,0.102365
6,50.0,0.00,Adaptive_Lasso,Sparse,0.053790,0.065450,0.061030
7,50.0,0.25,Adaptive_Lasso,Sparse,0.086425,0.100168,0.094649
8,50.0,0.50,Adaptive_Lasso,Sparse,0.138377,0.158457,0.150460
9,10.0,0.00,Adaptive_Lasso,Dense,0.087056,0.089959,0.087139


In [69]:
AdaRidge_results = AdaRidge_results.reset_index(drop = True)

In [70]:
AdaRidge_results

,p,rho,Estimator,Signal,AIC.MSE,BIC.MSE,LOOCV.MSE
0,10,0.00,Adaptive_Ridge,Sparse,0.027214,0.027214,0.027227
1,10,0.25,Adaptive_Ridge,Sparse,0.037419,0.037419,0.037549
2,10,0.50,Adaptive_Ridge,Sparse,0.050249,0.050249,0.050657
3,25,0.00,Adaptive_Ridge,Sparse,0.038296,0.038296,0.039878
4,25,0.25,Adaptive_Ridge,Sparse,0.056696,0.056707,0.060068
5,25,0.50,Adaptive_Ridge,Sparse,0.088347,0.088453,0.094249
6,50,0.00,Adaptive_Ridge,Sparse,0.038142,0.110648,0.045717
7,50,0.25,Adaptive_Ridge,Sparse,0.061805,0.296279,0.075238
8,50,0.50,Adaptive_Ridge,Sparse,0.104524,0.579844,0.126537
9,10,0.00,Adaptive_Ridge,Dense,0.090738,0.090738,0.090738


In [74]:
final_results = pd.concat([Lasso_results, AdaLasso_results, Ridge_results, AdaRidge_results])
final_results

,p,rho,Estimator,Signal,AIC.MSE,BIC.MSE,LOOCV.MSE
0,10.0,0.00,LASSO,Sparse,0.027627,0.027627,0.027703
1,10.0,0.25,LASSO,Sparse,0.037986,0.037986,0.038106
2,10.0,0.50,LASSO,Sparse,0.051029,0.051029,0.051203
3,25.0,0.00,LASSO,Sparse,0.042818,0.042818,0.043129
4,25.0,0.25,LASSO,Sparse,0.063914,0.063914,0.064389
...,...,...,...,...,...,...,...
13,25.0,0.25,Adaptive_Ridge,Dense,0.107668,0.107710,0.111255
14,25.0,0.50,Adaptive_Ridge,Dense,0.154900,0.155100,0.163434
15,50.0,0.00,Adaptive_Ridge,Dense,0.061254,0.351417,0.067009
16,50.0,0.25,Adaptive_Ridge,Dense,0.082762,0.465310,0.094700


In [75]:
final_results.to_csv('Final_results.csv')